In [8]:
import numpy as np
import pandas as pd
from functions import *

# load orbit data
juice_wrt_callisto_cphio = get_spice_data('juice', 'callisto', 'cphio', 'J')
callisto_wrt_jupiter_cphio = get_spice_data("callisto", "jupiter", "cphio", "J")
callisto_wrt_jupiter_JSO = get_spice_data("callisto", "jupiter", "jupsunorb", "J")
sun_wrt_callisto_cphio = get_spice_data("sun", "callisto", "cphio", "J")
callisto_wrt_jupiter_SIII_mag = get_spice_data("callisto", "jupiter", "SIII_mag", "J")
jupiter_wrt_sun_IAU = get_spice_data('jupiter', 'sun', 'IAU_SUN', 'J')

# save CA data
juice_wrt_callisto_cphio_CA, sun_wrt_callisto_cphio_CA, callisto_wrt_jupiter_JSO_CA, callisto_wrt_jupiter_SIII_mag_CA, jupiter_wrt_sun_IAU_CA = closest_approach_data_5(juice_wrt_callisto_cphio, sun_wrt_callisto_cphio, callisto_wrt_jupiter_JSO, callisto_wrt_jupiter_SIII_mag, jupiter_wrt_sun_IAU)


# collects important data from the individual frames

important_info_jcalcphio = np.array(['theta (CPhio)', 'phi (CPhiO)'])
for orbit, vector in juice_wrt_callisto_cphio_CA.items():
    important_info_i = [vector[5], vector[6]]
    important_info_jcalcphio = np.c_[important_info_jcalcphio, important_info_i]

important_info_caljupjso = np.array(['phi (JSO)'])
for orbit, vector in callisto_wrt_jupiter_JSO_CA.items():
    important_info_i = [vector[6]]
    important_info_caljupjso = np.c_[important_info_caljupjso, important_info_i]

important_info_caljupS3mag = np.array(['z (SIII Mag)'])
for orbit, vector in callisto_wrt_jupiter_SIII_mag_CA.items():
    important_info_i = [vector[3]]
    important_info_caljupS3mag = np.c_[important_info_caljupS3mag, important_info_i]

important_info_jupsunIAU = np.array(['phi (IAU_SUN)'])
for orbit, vector in jupiter_wrt_sun_IAU_CA.items():
    important_info_i = [vector[6]]
    important_info_jupsunIAU = np.c_[important_info_jupsunIAU, important_info_i]

important_info_jsun_daynightside_angle = np.array('psi (Day-Night)')
for i in range(len(juice_wrt_callisto_cphio_CA)):
    jcal_vector = juice_wrt_callisto_cphio_CA['CA_orbit%s' %(i+1)]
    suncal_vector = jupiter_wrt_sun_IAU_CA['CA_orbit%s' %(i+1)]
    daynightside_angle = [angle_between(jcal_vector[1:4], suncal_vector[1:4])]
    important_info_jsun_daynightside_angle = np.c_[important_info_jsun_daynightside_angle, daynightside_angle]

# collects all important data into one array
important_data = np.r_[important_info_jcalcphio, important_info_caljupjso]
important_data = np.r_[important_data, important_info_caljupS3mag]
important_data = np.r_[important_data, important_info_jupsunIAU]
important_data = np.r_[important_data, important_info_jsun_daynightside_angle]
important_data = np.transpose(important_data)
attributes = important_data[0,:]

# scales z (SIII Mag) to R_J
a = important_data[1:, 3]
a = a.astype(float)
a = a / np.array(R_J)
important_data[1:, 3] = a

# converts angles to degrees
b = important_data[1:, :3].astype(float)
c = important_data[1:, 4:].astype(float)

b = (b * np.array(180))/ np.array(np.pi)
c = (c * np.array(180))/ np.array(np.pi)

important_data[1:, :3] = b
important_data[1:, 4:] = c


KeyError: 'CA_orbit21'

In [ ]:
# sets limits for the graphs
theta_cphio_lims = [-13.5, 193.5] ; theta_cphio_ticks = [0, 90, 180]
z_S3Mag_lims = [-5.375,5.375] ; z_S3Mag_ticks = [-5, -2, 0, 2, 5]
daynightside_lims = [-13.5, 193.5] ; daynightside_ticks = [0, 90, 180]
phi_lims = [-205, 205] ; phi_ticks = [-180, 0, 180]
lims = [theta_cphio_lims, phi_lims, phi_lims, z_S3Mag_lims, phi_lims, daynightside_lims]
ticks = [theta_cphio_ticks, phi_ticks, phi_ticks, z_S3Mag_ticks, phi_ticks, daynightside_ticks]
# plots and saves all graphs from 2 - 21 orbits to generate gif of coverage progression

all_graphs = -19
final_graph = 0

for k in range(final_graph, 1):
    if k == 0:
        important_data_i = important_data[1:,:]
    else:
        important_data_i = important_data[1:k,:]
    df = pd.DataFrame(important_data_i, columns=attributes)
    df = df.astype(float)
    axs = pd.plotting.scatter_matrix(df, figsize=(12, 8), color='r')
    
    # sets limits for graphs
    for i in range(6):
        for j in range(6):
            axs[i,j].set_xlim(lims[j])
            axs[i,j].set_xticks(ticks[j])
            if i != j:
                axs[i,j].set_ylim(lims[i])
                axs[i,j].set_yticks(ticks[i])
    
    # draws grouping lines on graphs
    for i in range(6):
        for j in range(6):
            if i != j:
                if i == 0: 
                    axs[i,j].plot(lims[j], [90, 90], color='gray', linestyle='dashed', alpha=0.2)
                elif i == 1:
                    axs[i,j].plot(lims[j], [0,0], color='gray', linestyle='dashed', alpha=0.2)
                elif i == 2:
                    axs[i,j].plot(lims[j], [0,0], color='gray', linestyle='dashed', alpha=0.2)
                elif i == 3:
                    axs[i,j].plot(lims[j], [-2,-2], color='gray', linestyle='dashed', alpha=0.2)
                    axs[i,j].plot(lims[j], [ 2, 2], color='gray', linestyle='dashed', alpha=0.2)
                elif i == 4:
                    axs[i,j].plot(lims[j], [0,0], color='gray', linestyle='dashed', alpha=0.2)
                elif i == 5:
                    axs[i,j].plot(lims[j], [90, 90], color='gray', linestyle='dashed', alpha=0.2)
                
                if j == 0:
                    axs[i,j].plot([90, 90], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                elif j == 1:
                    axs[i,j].plot([0,0], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                elif j == 2:
                    axs[i,j].plot([0,0], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                elif j == 3:
                    axs[i,j].plot([-2,-2], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                    axs[i,j].plot([ 2, 2], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                elif j == 4:
                    axs[i,j].plot([0,0], lims[i], color='gray', linestyle='dashed', alpha=0.2)
                elif j == 5:
                    axs[i,j].plot([90, 90], lims[i], color='gray', linestyle='dashed', alpha=0.2)

               
    #plt.savefig('scatter_matrix_%s.png' % (21+k))
    plt.show()


NameError: name 'important_data' is not defined